In [ ]:
import os
import time

import win32file
import win32con
import pywintypes
import winerror
import win32process




filter_filetype =[".txt", ".exe", "hwp"]

def filterFile(filename):
    for type in filter_filetype:
        if filename.find(type) != -1:
            return True
    return False
    
    


WATCHED_DIR = "C:\\" # C 드라이브 내부 파일을 확인하기 위한 경로 설정

FILE_ACTIONS = {
  1 : "created",
  2 : "deleted",
  3 : "updated",
  4 : "renamed from",
  5 : "renamed to"
}

def main():
    ol = pywintypes.OVERLAPPED()
    buf = win32file.AllocateReadBuffer(1024)

    # 디렉토리
    hDir = win32file.CreateFile (
        WATCHED_DIR,                            # fileName
        0x0001, # FILE_LIST_DIRECTORY           # desiredAccess
        win32con.FILE_SHARE_READ |              # shareMode
        win32con.FILE_SHARE_WRITE | 
        win32con.FILE_SHARE_DELETE,
        None,                                   # attributes
        win32con.OPEN_EXISTING,                 # CreationDisposition
        win32con.FILE_FLAG_BACKUP_SEMANTICS |   # flagsAndAttributes
        win32con.FILE_FLAG_OVERLAPPED, # async
        None                                    # hTemplateFile
    )

    while True:
        # read directory changes async
        win32file.ReadDirectoryChangesW(
            hDir,   
            buf,    
            True,   
            win32con.FILE_NOTIFY_CHANGE_FILE_NAME | 
            win32con.FILE_NOTIFY_CHANGE_DIR_NAME |
            win32con.FILE_NOTIFY_CHANGE_ATTRIBUTES |
            win32con.FILE_NOTIFY_CHANGE_SIZE |
            win32con.FILE_NOTIFY_CHANGE_LAST_WRITE |
            win32con.FILE_NOTIFY_CHANGE_SECURITY,
            ol      
        )

        bytes = None

        # 에러처리
        while bytes == None:
            try:
                bytes = win32file.GetOverlappedResult(
                    hDir,   
                    ol,     
                    False   
                )

            except pywintypes.error as err:
                if err.winerror == winerror.ERROR_IO_INCOMPLETE:
                    time.sleep(0.1)

                else:
                    raise err

        
        result = win32file.FILE_NOTIFY_INFORMATION(buf, bytes)
        
        
        # 결과출력
        for action, filename in result:
            print("Alert! File Action Alert!!")
            if filterFile(filename):
                print(FILE_ACTIONS.get(action, "Unknown"), filename)
                if FILE_ACTIONS.get(action, "Unknown") == 'updated':
                    f = open(f"C:\\{filename}", "r")
                    while True:
                        line = f.readline()
                        if not line: break
                        print(line)
                    
                    f.close()


                
                print("---")
        #print("---")

if __name__ == "__main__":
    try:
        main()
        
    except Exception as e:
        print(e)